In [1]:
!pip install -qU crewai[tools,agentops]==0.95.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.9/211.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.0/134.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.9/542.9 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.3 MB/s eta 0:

In [2]:
pip install tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.3 MB/s eta 0:00:00


In [3]:
!pip install scrapegraph-py


In [4]:
from crewai import Agent, Task , Crew , Process ,LLM
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

import agentops
from pydantic import Field , BaseModel
from tavily import TavilyClient
from crewai.tools import tool
from scrapegraph_py import Client

from google.colab import userdata
import os

In [5]:
#os.environ['HUGGINGFACE_API_KEY'] = userdata.get('HUGGINGFACE_API_KEY')
#os.environ['DEEPSEEK_API_KEY'] = userdata.get('DEEPSEEK_API_KEY')
#os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')
os.environ['AGENTSAI_API_KEY'] = userdata.get('AGENTSAI_API_KEY')
os.environ['TAVILY_API_KEY']=userdata.get('TAVILY_API_KEY')



In [6]:
agentops.init(

              api_key=userdata.get('AGENTSAI_API_KEY'),
              skip_auto_end_session=True
)

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=5efdb67f-0735-49a1-ab46-7e02828317ad


In [7]:
output_dir = './agentops_output'
os.makedirs(output_dir, exist_ok=True)


basic_llm = LLM(
    model="gemini/gemini-1.5-pro-latest",
    api_key=userdata.get('GEMINI_API_KEY'),
    temperature=0
)

# Step 1. Instantiating your TavilyClient
tavily_client = TavilyClient(api_key=userdata.get('TAVILY_API_KEY'))

client = Client(api_key=userdata.get('SCRAP_API_KEY'))


In [8]:
no_queries=3
no_webs_names=3
about_company = "Art Decorations company work in home decorations and handmade and looking to buy Macrame at best prices (value for a price strategy)"
company_context = StringKnowledgeSource(
    content = about_company
)

# Setup Agents

# Agent A : Websites Recommender

In [9]:
class SuggestedWebsites(BaseModel):
  websites: list[str] = Field(...,title="suggested Websites that buying the product",   # the shape of queries I should identify
                              min_items=1,max_items=no_webs_names)

websites_recommender_agent=Agent(
    role='Websites names recommender agents',
    goal =   '\n'.join(["To provied a list of websites names that will be used to search for products.",
                        'this websites must be varied and sell the specifics products']),
    backstory='The agent is designed to help in looking for commercial websites where the product that I specify buying',
    llm = basic_llm,
    verbose=True,
)
websites_recommender_task = Task(
    description= '\n'.join([ "Art Decorations company is looking to buy {product_name} at best prices (value for a price strategy)",
                            'From your target websites, the company will search about the product with special search queries',
                             "The number of websites names must be at max {no_webs_names} names and be varied .",
                             "The websites may be popular or not",
                             "All websites must be aviable in {country_name} and prefare to work in products like {product_name} ",
    ]),
    expected_output='A JSON object containing a list of suggested Websites that buying the product',
    output_json=SuggestedWebsites,          # the shape of json file from the class
    output_file=os.path.join(output_dir,'step_0_suggested_websites_names.json'),           # the output dir that the json file will save in
    agent=websites_recommender_agent,

)

# Agent B: Queries Recommender

In [10]:
class SuggestedSearchQueries(BaseModel):
  queries: list[str] = Field(...,title="Suggested search queries to be passed to search engine.",   # the shape of queries I should identify
                              min_items=1,max_items=no_queries)


search_queries_recommender_agent=Agent(
    role='Search queries recommender agents',
    goal =   '\n'.join(['This agent wil take the websites names and provied a list of suggested search queries to be passed to search engine. ',
               "To provied a list of suggested search queries to be passed to search engine.",
               "The queries must be varied and locked to a specific items" ])  ,
    backstory="The agent is designed to helped in looking for products by suggesting a queries for this products to be passed to search engine.",
    llm = basic_llm,
    verbose=True,
)


search_queries_recommender_task = Task(
    description =f'\n'.join([
                             "Art Decorations company is looking to buy {product_name} at best prices (value for a price strategy)",
                             "From your target queries the company will search on the search engine.",
                             "The company will target products from the websites names that passed to it. ",
                             "The number of queried must be at max {no_queries} queries and be varied and locked to a specific items.",
                             "All products must be aviable in {country_name}",
                             "The queries must reach ecommerse webs, not blogs or listing pages"
    ]),

    expected_output='A JSON object containing a list of suggested queries to be passed to search engine.',
    output_json=SuggestedSearchQueries,          # the shape of json file from the class
    output_file=os.path.join(output_dir,'step_1_suggested_search_queries.json'),           # the output dir that the json file will save in
    agent=search_queries_recommender_agent,

)

**role** => the title of agent job  
**goal** => what the agent should do   
**backstory** => describe the job of the agent and what it should do in some details

**Task** => identify the task in details   
**description** => describe the task with details

# Agent C : Search Engine
Second agent will take the queries as inputs and get URLs to pages

In [11]:
th_score =0.5
class SingleResultSearchEngine(BaseModel):
    title: str
    URL: str
    content_and_information: str
    score: float
    query: str


class SearchEngineResults(BaseModel):
    results: list[SingleResultSearchEngine]


@tool
def search_engine_tool(query: str):
    """Search for product information using search engines, returning structured results."""
    response = tavily_client.search(query)

    # Convert response into the expected format (pseudo-code, adjust as needed)
    structured_results = []
    for item in response["results"]:
        structured_results.append(SingleResultSearchEngine(
            title=item.get("title", ""),
            URL=item.get("URL", ""),
            content_and_information=item.get("snippet", ""),
            score=item.get("score", 0.0),
            query=query
        ))

    return SearchEngineResults(results=structured_results)


search_engine_agent = Agent(
    role="Search Engine Agent",
    goal="Find relevant product pages from commercial websites.",
    backstory="\n".join([
        "This agent retrieves product URLs and information from commercial sources.",
        "It filters out non-commercial sources like blogs, academic pages, and discussion forums."
    ]),
    llm=basic_llm,
    verbose=True,
    tools=[search_engine_tool]
)

search_engine_task = Task(
    description="\n".join([
        "Retrieve product pages using search engines.",
        f"Return exactly {no_queries} product pages.",
        "Extract product details, pricing, and availability where possible.",
        "Only use commercial sources (e.g., e-commerce platforms, manufacturer websites).",
        f"Ignore results with confidence scores lower than {th_score}.",
        "Ensure results point to product pages, not general listings.",
        "Save results as a JSON file for further processing."
    ]),
    expected_output="A JSON object containing a list of commercial website URLs and product details.",
    output_json=SearchEngineResults,
    output_file=os.path.join(output_dir, 'step_2_suggested_search_engine_response.json'),
    agent=search_engine_agent,
    prevent_redundant_inputs=True
)


# Agent D :Web Scrapper
here we will use a tool to extract the informations from websites the Agent C suggested  
We need a tool to scrap the websites so we will use ScrapeGraphAI.  


In [12]:
class ProductSpecification(BaseModel):
      specification_name : str = Field(...,title = 'the name of the specification')
      specification_value : str = Field(...,title = 'the value of the specification')

class ProductDetails(BaseModel):
      product_url: str = Field(..., title='the URL of product page' )
      product_title : str = Field(...,title = 'The title of the product.')
      product_image_url : str = Field(...,title = 'the URL of the product image')
      product_current_price : float= Field(title = 'the price of the product')
      product_original_price : float = Field(title = 'the original price of the product',default=None)
      product_discount_percentage : float = Field(title = 'the discount percentage of the product',default=None)

      product_specifications : list[ProductSpecification] = Field(...,title = 'the specifications of the product focus on most important specifications',min_items = 1 ,max_items = 6)


      agent_recommendation_rank : float = Field(...,title = 'The rank of the product to be considered in the final report (out of 5 and Higher is Better) in the recommendation list order from better to worest.' )
      agent_recommendation_notes : list[str] = Field(...,title = 'A set of notes why you recommended and not recommended the product in this rank')



class AllProductsDetails(BaseModel):
  products : list[ProductDetails]



@tool
def scrapping_tool(url:str):
  '''
    An AI tool to scrapping informations and details from websites
    Example:
    scrapping_tool(url:"https://www.amazon.eg/-/en/Osunnus-Macrame-Curtain-Bohemian-Backdrop/dp/B0B7ND95X9",
    required_prompts : ['Price','Details','seller','shipping'])

  '''
  details = client.smartscraper(
    website_url=url,
    user_prompt = "Extract```json\n"+ ProductDetails.schema_json() +  "```\n from this website",
    )

  return {
      'Page URL':url,
      'Details':details
    }


scraping_agent = Agent(
    role='scraping information from websites.',
    goal='To scrape information and details about products from websites',
    backstory = '\n'.join(['this agent will take the URLs and maybe a small content and go to this product page and scripting and extract the informations about this product',
                           'This details wil use to decided which product is petter.']),
    llm = basic_llm,
    verbose=True,
    tools = [scrapping_tool]
)

scrabting_task = Task(
    description = '\n'.join(['This agent will take the URLs and maybe a small content and go to this product page and scripting and extract the informations about this product',
                             'Will return informations for number of pages equal to the number of URLS that passed to it.',
                           'This details wil use to decided which product is petter.',
                             'Collect the best 10 {no_best_products} from the search result',
                             'I need details about products.'
    ]),
    expected_output='A JSON object containing products details',
     output_json=AllProductsDetails,
     output_file=os.path.join(output_dir,'step_3_scraping_products_details.json'),           # the output dir that the json file will save in
     agent=scraping_agent,

)

# Agent D : Report Writer

In [13]:
report_writer_agent = Agent(
    role='Procurement Report Writer',
    goal ='To generate a professional HTML page for the procurement report.',
    backstory = 'This agent will take the details of products and create a HTML page for the procurement report.',
    llm = basic_llm,
    vabose=True,
)

report_writer_task =Task(
    description='\n'.join(['This agent will create a professional HTML page for procurement report.',
                           'You have to use Bootstrab CSS for a professional look and feel.',
                           'Use the provided context about the company to make a specialized report',
                           'The report will include products prices and details',
                           'The report should be sructured to the following output:',
                           'Title & Introduction :A clear title for the report (e.g., "Product Comparison Report").,A brief introduction explaining the purpose of the comparison.',
                           'Product Overview: A table or grid layout displaying key product details (e.g., name, price, specifications, discount, ratings)., Hyperlinks to the original product pages.',
                           'Analysis & Insights: A detailed breakdown of product strengths and weaknesses. Key trends and comparisons are highlighted using visual elements such as charts, icons, or color-coded indicators.',
                           'Recommendations: Suggested best product based on different criteria such as price, features, and customer reviews. Alternative recommendations are provided for different user needs (e.g., budget-friendly, high-end).',
                           'Conclusion: A summary of the findings and final thoughts on the best choice based on the collected data.',
                           'Additional Notes & References: Disclaimers, sources of data, and any additional references related to the product comparison.'

    ]),

    expected_output= 'A professional HTML page for the procurement report.',
    output_file=os.path.join(output_dir,'step_4_procurement_report.html'),
    agent=report_writer_agent,
    )





# Kickoff the Crew

In [14]:
crew = Crew(
    agents=[websites_recommender_agent,
            search_queries_recommender_agent,
            search_engine_agent,
            scraping_agent,
            report_writer_agent
            ],
    tasks=[websites_recommender_task,
           search_queries_recommender_task,
           search_engine_task,
           scrabting_task,
           report_writer_task
           ],
    process=Process.sequential , # or Process.hierarchical
    knowledge_sources=[company_context]
)


With premiume API the problem of qouta wil solve or we can use free models from Hugging Face

In [15]:
crew.kickoff(inputs={
    'product_name': "خيط مكرمية",
    'country_name':"Egypt",
    "no_queries" : 3,
    'topic': 'AI Agents',
    'th_score':0.2,
    'no_best_products':3,
    'no_webs_names':3

    })


# Agent: Websites names recommender agents
## Task: Art Decorations company is looking to buy خيط مكرمية at best prices (value for a price strategy)
From your target websites, the company will search about the product with special search queries
The number of websites names must be at max 3 names and be varied .
The websites may be popular or not
All websites must be aviable in Egypt and prefare to work in products like خيط مكرمية 


# Agent: Websites names recommender agents
## Final Answer: 
{
  "websites": ["souq.com", "jumia.com.eg", "amazon.eg"]
}


# Agent: Search queries recommender agents
## Task: Art Decorations company is looking to buy خيط مكرمية at best prices (value for a price strategy)
From your target queries the company will search on the search engine.
The company will target products from the websites names that passed to it. 
The number of queried must be at max 3 queries and be varied and locked to a specific items.
All products must be aviable in Egypt
The queries

ERROR:root:LiteLLM call failed: litellm.RateLimitError: litellm.RateLimitError: VertexAIException - {
  "error": {
    "code": 429,
    "message": "Resource has been exhausted (e.g. check quota).",
    "status": "RESOURCE_EXHAUSTED"
  }
}





LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

# Agent: Search Engine Agent
## Task: Retrieve product pages using search engines.
Return exactly 3 product pages.
Extract product details, pricing, and availability where possible.
Only use commercial sources (e.g., e-commerce platforms, manufacturer websites).
Ignore results with confidence scores lower than 0.5.
Ensure results point to product pages, not general listings.
Save results as a JSON file for further processing.


ERROR:root:LiteLLM call failed: litellm.RateLimitError: litellm.RateLimitError: VertexAIException - {
  "error": {
    "code": 429,
    "message": "Resource has been exhausted (e.g. check quota).",
    "status": "RESOURCE_EXHAUSTED"
  }
}





LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

# Agent: Search Engine Agent
## Task: Retrieve product pages using search engines.
Return exactly 3 product pages.
Extract product details, pricing, and availability where possible.
Only use commercial sources (e.g., e-commerce platforms, manufacturer websites).
Ignore results with confidence scores lower than 0.5.
Ensure results point to product pages, not general listings.
Save results as a JSON file for further processing.


ERROR:root:LiteLLM call failed: litellm.RateLimitError: litellm.RateLimitError: VertexAIException - {
  "error": {
    "code": 429,
    "message": "Resource has been exhausted (e.g. check quota).",
    "status": "RESOURCE_EXHAUSTED"
  }
}





LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers



RateLimitError: litellm.RateLimitError: litellm.RateLimitError: VertexAIException - {
  "error": {
    "code": 429,
    "message": "Resource has been exhausted (e.g. check quota).",
    "status": "RESOURCE_EXHAUSTED"
  }
}
